In [1]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path: sys.path.append("..")

# Segmentation and boundary-box regression

In [2]:
from src.utils.preprocess import tensor2numpy, image2axial,mask_threshold
from src.utils.utils import split_custom_filename
from src.segmentation import cam
from src.classifier.agent import load_trainer
from src.segmentation.cam import CAMS
from src.segmentation import segmentation
import nibabel as nib


import numpy as np
import torchvision
import torch
import matplotlib.pyplot as plt

In [3]:
for x,y in {'AD': [[17, 18, 79, 94, 2, 76]]}.items():
    print(x,y)

AD [[17, 18, 79, 94, 2, 76]]


In [4]:
from torchvision.ops._utils import convert_boxes_to_roi_format
derp = {key:convert_boxes_to_roi_format([torch.stack([torch.Tensor(x) for x in value])]).shape for key,value in {'AD': [[17, 18, 79, 94, 2, 76],[17, 18, 79, 94, 2, 76]]}.items()}
derp

{'AD': torch.Size([2, 7])}

## Train on ROI

In [5]:
from src.classifier import agent

roi_trainer,roi_dataset, roi_model =  agent.load_trainer('roi2');

# Ideas: Create a new dataloader which preprocess data for roi?
# * Basically insert ROIAlign in dataloading. Bad point: Not saved on disk
# * Save boundaries in some file to be loaded. When loading file run ROIAlign on them
# Boundaries needed for AD, MCI and CN
# 

Data directory: data/SPM_categorised/Amira/
Dataset sizes - Training: 500 Validation: 56


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Using native 16bit precision.


Architecture [ResNet] was created
What type is bounding boxes {2: [[17, 18, 79, 94, 2, 76]]} dict_items([(2, [[17, 18, 79, 94, 2, 76]])])
Some test here {2: tensor([[17., 18., 79., 94.,  2., 76.]])}
***Defined hyperparameters:***
"arch":          {'name': 'resnet50', 'args': {'num_channels': 1, 'num_classes': 3}}
"callbacks":     ['litprogressbar', 'modelcheckpoints', 'metriccallback']
"class_weights": tensor([1.1905, 0.9311, 0.9208], device='cuda:0')
"hp_metrics":    ['loss/train', 'loss/val', 'accuracy/val', 'auc/val', 'specificity/val', 'sensitivity/val']
"kfold":         {'enable': False, 'folds': 10}
"loss":          {'type': 'CrossEntropyLoss', 'args': {'weight': True}}
"optimizer":     {'type': 'Adam', 'args': {'lr': 0.0002, 'weight_decay': 1e-05, 'amsgrad': True}}
"roi_hparams":   {'enable': True, 'output_shape': [40, 40, 40], 'boundary_boxes': {2: [[17, 18, 79, 94, 2, 76]]}}


In [6]:
roi_trainer.fit(roi_model, datamodule=roi_dataset)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1

  | Name     | Type             | Params
----------------------------------------------
0 | model    | ResNet           | 46.2 M
1 | criteria | CrossEntropyLoss | 0     
----------------------------------------------
46.2 M    Trainable params
0         Non-trainable params
46.2 M    Total params
184.645   Total estimated model params size (MB)


KeyError: 1

In [ ]:
roi_trainer.predict()